In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import fnmatch
import errno    
import glob
from textwrap import dedent
from riboraptor.helpers import mkdir_p
from riboraptor.sradb import SRAdb
from riboraptor.utils import copy_sra_data, create_config_file

re_ribo_root_dir = '/staging/as/skchoudh/SRA_datasets/'
samples_to_process_dir = '/staging/as/skchoudh/re-ribo-datasets/'
re_ribo_config_dir = '/home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs'
out_dir = '/staging/as/skchoudh/re-ribo-analysis'

datasets = {}
datasets['hg38'] = pd.read_excel('../data/datasets/re-ribo-datasets.xlsx', sheet_name='hg38')
datasets['mm10'] = pd.read_excel('../data/datasets/re-ribo-datasets.xlsx', sheet_name='mm10')

Populating the interactive namespace from numpy and matplotlib


/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
datasets['hg38'].sort_values(by=['Tissue'])

,Publication,SRP,Status,PMID,Tissue
8,Loayza-Puch et al. (2013),SRP020544,NaN,23594524.0,BJ fibroblast
10,Rooijers et al. (2013),SRP027514,NaN,24301020.0,BJ fibroblast
3,Gonzalez et al. (2014),SRP031501,NaN,25122893.0,Brain
6,Lee et al. (2012),SRP014629,NaN,22927429.0,HEK293
7,Liu et al. (2012),SRP017263,NaN,23290916.0,HEK293
9,Reid and Nicchitta (2012),SRP007963,NaN,22199352.0,HEK293
0,Andreev et al. (2014),SRP038695,Done,25621764.0,HEK293T
4,Guo et al. (2010),SRP002605,NaN,20703300.0,HeLa
11,Stadler & Fire (2011),SRP010374,NaN,22045228.0,HeLa
13,Stumpf et al. (2013),SRP029589,NaN,24120665.0,HeLa


In [3]:
datasets['mm10'].sort_values(by=['Tissue'])

,Publication,SRP,Status,PMID,Tissue
6,Eichhorn et al. (2014),SRP045501,NaN,25263593.0,3T3/HEK293
7,Fields et al. (2015),SRP064972,NaN,26638175.0,Bone marrow-derived dendritic cells
15,Ingolia et al. (2011),SRP007567,NaN,22056041.0,E14
18,Laguesse et al. (2015),SRP065804,NaN,26651292.0,E14.5 forebrain
0,AlvarezDominguez et al. (2017),SRP077522,Started,27899360.0,E14.5 mouse fetal livers
16,Ingolia et al. (2014),SRP045257,NaN,25159147.0,ESC
14,Hurt et al. (2013),SRP016625,NaN,23766421.0,ESC
24,You et al. (2015),SRP064129,NaN,26443847.0,ESC
5,Cho et al. (2015),SRP062407,NaN,26430118.0,Hippocampal tissue
11,Guo et al. (2010),SRP003554,NaN,20703300.0,Neutrophils from bone marrow


In [4]:
db = SRAdb('/staging/as/skchoudh/SRAmetadb.sqlite')


In [10]:
srp = 'SRP055707'
df = db.sra_convert(srp)
#copy_sra_data(df)
#create_config_file(df=df)
df

,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP055707,SRX894007,SRR1822445,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,2588329650,51766593,None,50.0
1,SRP055707,SRX894008,SRR1822446,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,3450906650,69018133,None,50.0
2,SRP055707,SRX894009,SRR1822447,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,2983328950,59666579,None,50.0
3,SRP055707,SRX894010,SRR1822448,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,3659227900,73184558,None,50.0
4,SRP055707,SRX894011,SRR1822449,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,867903050,17358061,None,50.0
5,SRP055707,SRX894012,SRR1822450,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1254858150,25097163,None,50.0
6,SRP055707,SRX894013,SRR1822451,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,893037500,17860750,None,50.0
7,SRP055707,SRX894014,SRR1822452,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1287696900,25753938,None,50.0
8,SRP055707,SRX894015,SRR1822453,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1156349950,23126999,None,50.0
9,SRP055707,SRX894016,SRR1822454,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1275071300,25501426,None,50.0


In [9]:
srp = 'SRP056647'
df = db.sra_convert(srp)
df = df[df.taxon_id==4932]
copy_sra_data(df)
create_config_file(df=df)
df


100%|██████████| 90/90 [00:00<00:00, 267.51it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/sacCerR64_SRP056647.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP056647,SRX971707,SRR1944937,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,588362136,20126894,None,29.232635
1,SRP056647,SRX971683,SRR1944913,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1095851943,37334344,None,29.352382
2,SRP056647,SRX971682,SRR1944912,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,855004779,28905052,None,29.579770
3,SRP056647,SRX971708,SRR1944938,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,889343727,29893433,None,29.750472
4,SRP056647,SRX971706,SRR1944936,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,680664501,22825808,None,29.819952
5,SRP056647,SRX971685,SRR1944915,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,970021648,32467816,None,29.876406
6,SRP056647,SRX971761,SRR1944993,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,449713788,14979681,None,30.021586
7,SRP056647,SRX971684,SRR1944914,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,525959616,17505437,None,30.045500
8,SRP056647,SRX971747,SRR1944979,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1216625176,40491503,None,30.046432
9,SRP056647,SRX971755,SRR1944987,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,155111036,5157425,None,30.075287


In [8]:
srp = 'SRP038695'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df


100%|██████████| 12/12 [00:00<00:00, 28.17it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP038695.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP038695,SRX474109,SRR1173905,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,794041050,24061850,None,33.0
1,SRP038695,SRX474110,SRR1173906,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,285938268,8664796,None,33.0
2,SRP038695,SRX474111,SRR1173907,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,783682548,23747956,None,33.0
3,SRP038695,SRX474112,SRR1173908,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,268895682,8148354,None,33.0
4,SRP038695,SRX474113,SRR1173909,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,4573442293,93335557,None,49.0
5,SRP038695,SRX474114,SRR1173911,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1281109508,26145092,None,49.0
6,SRP038695,SRX474115,SRR1173913,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,5910143428,120615172,None,49.0
7,SRP038695,SRX474116,SRR1173915,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1433175177,29248473,None,49.0
8,SRP038695,SRX474113,SRR1173910,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,645383400,8605112,None,75.0
9,SRP038695,SRX474114,SRR1173912,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,294934875,3932465,None,75.0


In [6]:
srp = 'SRP002605'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df


100%|██████████| 24/24 [00:00<00:00, 77.16it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP002605.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP002605,SRX021966,SRR057511,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546920: Footprint_mock_32hr_runs1-2,173069316,4807481,None,36.0
1,SRP002605,SRX021966,SRR057512,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546920: Footprint_mock_32hr_runs1-2,670274352,18618732,None,36.0
2,SRP002605,SRX021967,SRR057513,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546921: mRNASeq_mock_32hr_runs1-3,159276276,4424341,None,36.0
3,SRP002605,SRX021967,SRR057515,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546921: mRNASeq_mock_32hr_runs1-3,92929104,2581364,None,36.0
4,SRP002605,SRX021968,SRR057516,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546922: Footprint_miR-155_32hr_runs1-2,212446440,5901290,None,36.0
5,SRP002605,SRX021968,SRR057517,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546922: Footprint_miR-155_32hr_runs1-2,522725688,14520158,None,36.0
6,SRP002605,SRX021969,SRR057518,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546923: mRNASeq_miR-155_32hr_runs1-3,216374004,6010389,None,36.0
7,SRP002605,SRX021969,SRR057519,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546923: mRNASeq_miR-155_32hr_runs1-3,207243972,5756777,None,36.0
8,SRP002605,SRX021969,SRR057520,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546923: mRNASeq_miR-155_32hr_runs1-3,94523436,2625651,None,36.0
9,SRP002605,SRX021970,SRR057521,9606,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546924: Footprint_miR-1_32hr_runs1-2,212789088,5910808,None,36.0


In [7]:
srp = 'SRP003554'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df


100%|██████████| 10/10 [00:00<00:00, 96.11it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP003554.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP003554,SRX026871,SRR065774,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546987: Footprint_wild-type_runs1-2,220030992,6111972,None,36.0
1,SRP003554,SRX026871,SRR065775,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546987: Footprint_wild-type_runs1-2,466656444,12962679,None,36.0
2,SRP003554,SRX026872,SRR065776,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546988: mRNASeq_wild-type_runs1-3,205130160,5698060,None,36.0
3,SRP003554,SRX026872,SRR065777,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546988: mRNASeq_wild-type_runs1-3,428629788,11906383,None,36.0
4,SRP003554,SRX026872,SRR065778,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546988: mRNASeq_wild-type_runs1-3,62993088,1749808,None,36.0
5,SRP003554,SRX026873,SRR065779,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546989: Footprint_mir-223-knockout_runs1-2,211817628,5883823,None,36.0
6,SRP003554,SRX026873,SRR065780,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546989: Footprint_mir-223-knockout_runs1-2,549140364,15253899,None,36.0
7,SRP003554,SRX026874,SRR065781,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546990: mRNASeq_mir-223-knockout_runs1-3,171575640,4765990,None,36.0
8,SRP003554,SRX026874,SRR065782,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546990: mRNASeq_mir-223-knockout_runs1-3,490349304,13620814,None,36.0
9,SRP003554,SRX026874,SRR065783,10090,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,GSM546990: mRNASeq_mir-223-knockout_runs1-3,97069104,2696364,None,36.0


In [6]:
srp = 'SRP049168'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 12/12 [00:00<00:00, 66.27it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP049168.py
Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP049168.py


In [7]:
srp = 'SRP074765'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 82/82 [00:00<00:00, 70.03it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP074765.py


In [10]:
srp = 'SRP064304'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 884/884 [00:03<00:00, 251.67it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP064304.py


In [11]:
srp = 'SRP078005'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 18/18 [00:00<00:00, 61.26it/s]


Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP078005.py


In [18]:
srp = 'SRP021058'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 12/12 [00:00<00:00, 144.59it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP021058.py


In [15]:
srp = 'SRP076093'
df = db.sra_convert(srp)
df

,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP076093,SRX1818687,SRR3623562,10090,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1273893500,25477870,None,50.0
1,SRP076093,SRX1818688,SRR3623563,10090,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1327949700,26558994,None,50.0
2,SRP076093,SRX1818689,SRR3623564,10090,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1501223250,30024465,None,50.0
3,SRP076093,SRX1818682,SRR3623557,1247190,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1290162950,25803259,None,50.0
4,SRP076093,SRX1818683,SRR3623558,1247190,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,961952850,19239057,None,50.0
5,SRP076093,SRX1818684,SRR3623559,1247190,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1061998750,21239975,None,50.0
6,SRP076093,SRX1818685,SRR3623560,1247190,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1201295750,24025915,None,50.0
7,SRP076093,SRX1818686,SRR3623561,1247190,size fractionation,SINGLE -,ncRNA-Seq,TRANSCRIPTOMIC,None,1108448800,22168976,None,50.0


In [17]:
srp = 'SRP076093'
df = db.sra_convert(srp)
#copy_sra_data(df)
create_config_file(df=df)

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP076093.py


KeyError: 1247190

In [14]:
srp = 'SRP056576'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 56/56 [00:00<00:00, 64.65it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP056576.py


In [16]:
srp = 'SRP030014'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 19/19 [00:00<00:00, 55.40it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP030014.py


In [23]:
srp = 'SRP043036'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df



100%|██████████| 12/12 [00:00<00:00, 108.21it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/sacCerR64_SRP043036.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP043036,SRX581792,SRR1363414,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1957041936,54362276,None,36.0
1,SRP043036,SRX581797,SRR1363419,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1435095972,39863777,None,36.0
2,SRP043036,SRX581798,SRR1363420,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1078644888,29962358,None,36.0
3,SRP043036,SRX581798,SRR1363421,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1430761104,39743364,None,36.0
4,SRP043036,SRX581799,SRR1363422,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1487820708,41328353,None,36.0
5,SRP043036,SRX581799,SRR1363423,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1635037848,45417718,None,36.0
6,SRP043036,SRX581790,SRR1363412,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1734374300,34687486,None,50.0
7,SRP043036,SRX581791,SRR1363413,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1791908950,35838179,None,50.0
8,SRP043036,SRX581793,SRR1363415,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1855938900,37118778,None,50.0
9,SRP043036,SRX581795,SRR1363417,4932,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1708898250,34177965,None,50.0


In [25]:
srp = 'SRP063562'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df



100%|██████████| 5/5 [00:00<00:00, 71.09it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/MG1655_SRP063562.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP063562,SRX1219745,SRR2340141,511145,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1967578900,39351578,None,50.0
1,SRP063562,SRX1219746,SRR2340143,511145,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1958851700,39177034,None,50.0
2,SRP063562,SRX1219747,SRR2340144,511145,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,918302050,18366041,None,50.0
3,SRP063562,SRX1219748,SRR2340146,511145,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1430785650,28615713,None,50.0
4,SRP063562,SRX1219749,SRR2340147,511145,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,1520846350,30416927,None,50.0


In [28]:
srp = 'SRP098789'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 26/26 [00:00<00:00, 109.95it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP098789.py


In [27]:
df

,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP098789,SRX2536403,SRR5227288,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2104142750,42082855,None,50.0
1,SRP098789,SRX2536404,SRR5227289,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2082873050,41657461,None,50.0
2,SRP098789,SRX2536405,SRR5227290,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2023148650,40462973,None,50.0
3,SRP098789,SRX2536406,SRR5227291,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2057165950,41143319,None,50.0
4,SRP098789,SRX2536407,SRR5227292,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3027621850,60552437,None,50.0
5,SRP098789,SRX2536408,SRR5227293,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2135456900,42709138,None,50.0
6,SRP098789,SRX2536409,SRR5227294,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3800106100,76002122,None,50.0
7,SRP098789,SRX2536410,SRR5227295,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2306857400,46137148,None,50.0
8,SRP098789,SRX2536411,SRR5227296,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,2636889200,52737784,None,50.0
9,SRP098789,SRX2536412,SRR5227297,9606,other,SINGLE -,OTHER,TRANSCRIPTOMIC,None,3019068250,60381365,None,50.0


In [30]:
srp = 'SRP017942'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 11/11 [00:00<00:00, 33.19it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/hg38_SRP017942.py
Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/mm10_SRP017942.py


In [37]:
srp = 'SRP045475'
df = db.sra_convert(srp)
copy_sra_data(df)
create_config_file(df=df)
df

100%|██████████| 6/6 [00:00<00:00, 251.68it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/BDGP6_SRP045475.py


,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP045475,SRX679371,SRR1548659,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,9481573800,189631476,None,50.0
1,SRP045475,SRX679370,SRR1548658,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,718697100,14092100,None,51.0
2,SRP045475,SRX679368,SRR1548656,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,9997319700,99973197,None,100.0
3,SRP045475,SRX679369,SRR1548657,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,6723088500,67230885,None,100.0
4,SRP045475,SRX679372,SRR1548660,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,373767700,3737677,None,100.0
5,SRP045475,SRX679372,SRR1548661,7227,cDNA,SINGLE -,RNA-Seq,TRANSCRIPTOMIC,None,868997500,8689975,None,100.0


In [41]:
srp = 'SRP108999'
df = db.sra_convert(srp)
df['taxon_id'] = 7227
df['library_strategy'] = 'SINGLE'
copy_sra_data(df)
create_config_file(df=df)

100%|██████████| 21/21 [00:00<00:00, 51.63it/s]

Wrote /home/cmb-panasas2/skchoudh/github_projects/re-ribo-smk/snakemake/configs/BDGP6_SRP108999.py


In [42]:
df

,study_accession,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,bases,spots,adapter_spec,avg_read_length
0,SRP108999,SRX2902866,SRR5667278,7227,None,None,SINGLE,None,None,676573468,20752434,None,32.602126
1,SRP108999,SRX2902865,SRR5667277,7227,None,None,SINGLE,None,None,2610438767,78959629,None,33.060423
2,SRP108999,SRX2902871,SRR5667283,7227,None,None,SINGLE,None,None,1104998701,32872272,None,33.614917
3,SRP108999,SRX2902864,SRR5667276,7227,None,None,SINGLE,None,None,1194135764,35199688,None,33.924612
4,SRP108999,SRX2902857,SRR5667269,7227,None,None,SINGLE,None,None,1539402070,45156367,None,34.090477
5,SRP108999,SRX2902856,SRR5667268,7227,None,None,SINGLE,None,None,4216803822,122734505,None,34.357118
6,SRP108999,SRX2902870,SRR5667282,7227,None,None,SINGLE,None,None,1940386794,56343352,None,34.438611
7,SRP108999,SRX2902872,SRR5667284,7227,None,None,SINGLE,None,None,2392506309,69053087,None,34.647348
8,SRP108999,SRX2902855,SRR5667267,7227,None,None,SINGLE,None,None,1621851282,46651854,None,34.764991
9,SRP108999,SRX2902861,SRR5667273,7227,None,None,SINGLE,None,None,1408728811,20062370,None,70.217467
